In [48]:
import openai
from dotenv import load_dotenv
import os
import json

load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [49]:
filename = 'db-gesamt-dashboard'
import_filename = 'text/' + filename + '.txt'

# Load text file as plain text
with open(import_filename, encoding='utf-8') as f:
    text = f.read()

### Summary

In [55]:
delimiter = "####"

messages = [
    {"role": "system", "content": "You are a user researcher"},
    {"role": "user", "content": text}
]

messages = [
    {"role": "system", "content": """
        You are a user researcher. You analyze interviews with travellers of DB Deutsche Bahn. \
        You extract the Names of people interviewed
        
        *****
        """},
    {"role": "user", "content": text}
]

def get_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=messages,
    )
    content = response['choices'][0]['message']['content']

    # Print the content
    print(content)

    return response

get_completion(messages)

Christoph, Julia, Person x, Person y, Person z, Person p, Person n, Susanne, Gast, MCK, TJ, Frau (ca. 40 Jahre alt), Frau (ca. 40 Jahre alt), Mann (Anfang 40), Sybille, Elly & Farina mit Baby (6 Wochen alt), Farina, W. Bielesza, Maren, Frau um die 50, F. Nißlmüller

**Note**: Some individuals are referred to as "Person x" or "Person y" when their name was not provided.


<OpenAIObject chat.completion id=chatcmpl-7UwdOzdZSYdhRT6sXlbxZK5youflR at 0x10f0aa870> JSON: {
  "id": "chatcmpl-7UwdOzdZSYdhRT6sXlbxZK5youflR",
  "object": "chat.completion",
  "created": 1687609586,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Christoph, Julia, Person x, Person y, Person z, Person p, Person n, Susanne, Gast, MCK, TJ, Frau (ca. 40 Jahre alt), Frau (ca. 40 Jahre alt), Mann (Anfang 40), Sybille, Elly & Farina mit Baby (6 Wochen alt), Farina, W. Bielesza, Maren, Frau um die 50, F. Ni\u00dflm\u00fcller\n\n**Note**: Some individuals are referred to as \"Person x\" or \"Person y\" when their name was not provided."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16260,
    "completion_tokens": 123,
    "total_tokens": 16383
  }
}

### Structured data

In [35]:
messages = [
    {"role": "system", "content": "You are a user researcher"},
    {"role": "user", "content": text}
]

messages = [
    {"role": "system", "content": """
        You are a user researcher. You analyze interviews with travellers of DB Deutsche Bahn. \
        The interviews start below the separator *****
        You call functions to generate a dashboard \
        The first value you get is the number of persons interviewed. \
        Persons interviewed can be recognized by the pattern "name:" <question>. \
        Example: "Christian: <question>"
        
        The second thing you do is to generate a list of topics keywords raised by interviewees, that \
        would be relevant to a product manager improving the services. \
        After every keyword you add the number of occurencies in brackets.
        
        *****
        """},
    {"role": "user", "content": text}
]

def get_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=messages,
        functions= [
            {
              "name": "summarize_interviews",
              "description": "summarize ",
              "parameters": {
                "type": "object",
                "interviewees": {
                  "number of persons interviewed": {
                    "type": "string",
                    "description": "number of people interviewed"
                  },                   
                  "topics": {
                    "type": "string",
                    "description": "generate a list of topics keywords"
                  },
                }
              }
            }
        ],
        function_call="auto",
    )
    print(response)
    arguments = response['choices'][0]['message']['function_call']['arguments']
    arguments_dict = json.loads(arguments)
    return arguments_dict

get_completion(messages)


TypeError: __str__ returned non-string (type list)

### From deeplearning training
https://learn.deeplearning.ai/chatgpt-building-system/lesson/3/classification

In [58]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [59]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "Account Management",
  "secondary": "Close account"
}
